In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\KIIT\Videos\Face_Mask_Detection\face_detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\KIIT\Videos\Face_Mask_Detection\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [4]:
model=load_model(r'C:\Users\KIIT\Videos\Face_Mask_Detection\resnet50.model')

In [5]:
image=cv2.imread(r'C:\Users\KIIT\Videos\Face_Mask_Detection\example\old.jpg')

In [6]:
image

array([[[238, 240, 240],
        [253, 255, 255],
        [241, 243, 243],
        ...,
        [254, 251, 253],
        [252, 251, 255],
        [252, 251, 255]],

       [[251, 253, 253],
        [253, 255, 255],
        [253, 255, 255],
        ...,
        [254, 251, 253],
        [252, 251, 255],
        [252, 251, 255]],

       [[238, 240, 240],
        [253, 255, 255],
        [251, 253, 253],
        ...,
        [254, 252, 252],
        [252, 251, 255],
        [252, 251, 255]],

       ...,

       [[250, 242, 243],
        [255, 251, 252],
        [250, 255, 255],
        ...,
        [246, 250, 251],
        [250, 250, 250],
        [250, 250, 250]],

       [[255, 248, 249],
        [255, 247, 247],
        [255, 253, 253],
        ...,
        [245, 250, 249],
        [249, 249, 249],
        [249, 249, 249]],

       [[255, 253, 252],
        [255, 253, 254],
        [255, 248, 248],
        ...,
        [246, 249, 247],
        [248, 248, 248],
        [248, 248, 248]]

In [7]:
(h,w)=image.shape[:2]
(h,w)

(183, 275)

In [8]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))
blob

array([[[[134., 147., 139., ..., 149., 148., 148.],
         [139., 148., 143., ..., 149., 148., 148.],
         [146., 149., 149., ..., 150., 148., 148.],
         ...,
         [151., 151., 151., ..., 142., 145., 145.],
         [151., 151., 151., ..., 142., 144., 144.],
         [151., 151., 151., ..., 142., 144., 144.]],

        [[ 63.,  76.,  68., ...,  74.,  74.,  74.],
         [ 68.,  77.,  72., ...,  74.,  74.,  74.],
         [ 75.,  78.,  78., ...,  74.,  74.,  74.],
         ...,
         [ 71.,  70.,  75., ...,  73.,  72.,  72.],
         [ 74.,  73.,  73., ...,  72.,  71.,  71.],
         [ 76.,  76.,  72., ...,  72.,  71.,  71.]],

        [[117., 130., 122., ..., 130., 132., 132.],
         [122., 131., 126., ..., 130., 132., 132.],
         [129., 132., 132., ..., 130., 132., 132.],
         ...,
         [126., 124., 129., ..., 126., 126., 126.],
         [128., 128., 127., ..., 125., 125., 125.],
         [129., 131., 126., ..., 124., 125., 125.]]]], dtype=float32)

In [9]:
net.setInput(blob)
detections=net.forward()

In [10]:
detections

array([[[[0.        , 1.        , 0.9982998 , ..., 0.19446346,
          0.36154372, 0.74171877],
         [0.        , 1.        , 0.9957416 , ..., 0.1940113 ,
          0.8708693 , 0.78570616],
         [0.        , 1.        , 0.11922034, ..., 0.31953982,
          0.8691126 , 0.4846194 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [11]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX),min(h-1,endY))
        
        face=image[startY:endY , startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else(0,0,255)
        
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
        